# Sign Up to Use Google Earth Engine (EE)

## Developer API

[Google Earth Engine sign up form](https://signup.earthengine.google.com/#!/)

Use of the service is strictly non-commercial. Please consult the T&Cs:

> 1.2 Limitations on Use. Unless otherwise approved by Google in writing, Services may only be used for development, research, or educational purposes; you can use data or figures created by use of Services in research or educational publications authored by you. Services may not be used for sustained commercial purposes unless otherwise approved by Google in writing, but may be evaluated in a production environment. Applications and data created with Services may not be sold or licensed for a fee unless otherwise approved by Google in writing.

[source: Google Earth Engine Terms of Use](https://earthengine.google.com/terms/)

As I'm logged into my Google profile, the form would only accept my gmail address. I selected affiliated with ONS and purpose was to 'investigate land use classification within my locality and nationally'. 

Notification to keep an eye on Gmail inbox over the next week for status update.

I had the confirmation Email come through within the hour, but I have a vague recollection of going through Earth Engine access before.

### Service Account

After reading the documentation and trying a couple of approaches, the method that worked for me was to configure my application as a [service account](https://developers.google.com/earth-engine/guides/service_account), which seems to be the method used for hosting of an application that can authenticate with the Earth Engine API for multiple end users. 

***

## Python client tutorial

Ensure the most recent release of earth engine client is being run with:
`pip install earthengine-api --upgrade`

Link to the [Google Earth Engine API tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)

Improve awareness of data available by browsing the [Earth Engine data catalogue](https://developers.google.com/earth-engine/datasets/catalog)

### EE-specific terms

The documentation refers to:

**Features**: Geometries. Like Spatial features.

**Images**: Rasters, tiffs and the like.

**Collections**: A resource that combines features & images.


### Tutorial Context

Resources used are:
* MODIS land cover (LC)
* MODIS land surface temp (LST)
* USGS ground elevation (ELV)

All three are `ee.ImageCollections`. Things we need from the dataset descriptions to work with these resources:
1. Availability
2. Provider
3. EE snippet
4. Available bands

So for [LC](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1):

![](../../www/modis_lc_metadata.png)


In [ ]:
import os

import ee
import toml
from pyprojroot import here

In [ ]:
service_email = toml.load(os.path.join(here(), ".git_ignore", "ee.toml"))["ee"][
    "SERVICE_EMAIL"
]

In [ ]:
credentials = ee.ServiceAccountCredentials(
    service_email,
    os.path.join(here(), ".git_ignore", "ee-lsoa-land-use-02b30d7b328b.json"),
)
ee.Initialize(credentials)

In [ ]:
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())